# License

In [3]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg
    #!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
    #!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz
installDeps()
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from audioInput import getLaughTracks, getNoise
slim = tf.contrib.slim

def trainAndSaveAndPredict(test_data, number_of_classes = 2, number_of_samples = 1, epochs = 5, getData = getLaughTracks, use_cache = True, log = True):
    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, use_cache = use_cache, log = log)
    model_name = 'model_%s_%s' % (number_of_samples, epochs)
    preds = train(curriedGetSamples, number_of_classes, model_name = model_name, epochs = epochs)
    

    return predict(model_name, number_of_classes, test_data)

def printResults(preds, expected = None): 
    with tf.Graph().as_default(), tf.Session() as sess:
        print(preds)
        print(sess.run(tf.argmax(input=preds, axis=1))) 
        print('expected results', expected)

def trainForNoise(number_of_samples=5, epochs=5):
    use_cache = False
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=5, epochs=5):  
    use_cache = False
    print('training on laughter and not laughter')
    (features, labels) = getLaughTracks(shuf=False, number_of_samples = 2, use_cache = use_cache, use_full_files = False, log=False)
    preds = trainAndSaveAndPredict(features, number_of_classes = 2, number_of_samples = number_of_samples, epochs = epochs, getData = getLaughTracks, use_cache = use_cache, log = False)
    printResults(preds, [0, 0, 1, 1])

/bin/sh: apt-get: command not found
--2018-09-23 18:28:31--  https://storage.googleapis.com/audioset/vggish_pca_params.npz
Resolving storage.googleapis.com... 172.217.12.208
Connecting to storage.googleapis.com|172.217.12.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73020 (71K) [application/octet-stream]
Saving to: 'vggish_pca_params.npz'

vggish_pca_params.n 100%[===================>]  71.31K  --.-KB/s    in 0.04s   

2018-09-23 18:28:31 (1.98 MB/s) - 'vggish_pca_params.npz' saved [73020/73020]



In [4]:
trainForLaughter(number_of_samples=1, epochs=1)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.691555
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/model_1_1
[[0.49445516 0.54533243]
 [0.43917602 0.61637205]
 [0.46065295 0.6182625 ]
 [0.4272686  0.6017852 ]]
[1 1 1 1]
expected results [0, 0, 1, 1]


In [5]:
trainForLaughter(number_of_samples=3, epochs=3)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.701443
Step 2: loss 0.678392
Step 3: loss 0.657077
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/model_3_3
[[0.59544694 0.39253926]
 [0.63829833 0.37715247]
 [0.5483389  0.47430268]
 [0.5759331  0.44201037]]
[0 0 0 0]
expected results [0, 0, 1, 1]


In [6]:
trainForLaughter(number_of_samples=5, epochs=5)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.711764
Step 2: loss 0.678519
Step 3: loss 0.665346
Step 4: loss 0.638338
Step 5: loss 0.619232
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/model_5_5
[[0.4671507  0.4948345 ]
 [0.57178277 0.4467245 ]
 [0.4347837  0.54266155]
 [0.4794064  0.54114735]]
[1 0 1 1]
expected results [0, 0, 1, 1]


In [7]:
trainForLaughter(number_of_samples=10, epochs=10)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.714611
Step 2: loss 0.692178
Step 3: loss 0.683827
Step 4: loss 0.674197
Step 5: loss 0.6581
Step 7: loss 0.624655
Step 8: loss 0.608556
Step 9: loss 0.59425
Step 10: loss 0.578849
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/model_10_10
[[0.62925553 0.42127648]
 [0.70041484 0.37787944]
 [0.5244432  0.52259976]
 [0.55212754 0.4606842 ]]
[0 0 0 0]
expected results [0, 0, 1, 1]


In [ ]:
trainForLaughter(number_of_samples=20, epochs=10)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.780964
Step 2: loss 0.734711
Step 3: loss 0.720537
Step 4: loss 0.71358
Step 5: loss 0.702921
Step 6: loss 0.689022


In [ ]:
trainForLaughter(number_of_samples=30, epochs=10)

In [ ]:
trainForLaughter(number_of_samples=40, epochs=10)

In [ ]:
trainForLaughter(number_of_samples=50, epochs=10)

In [ ]:
trainForLaughter(number_of_samples=50, epochs=50)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt


In [ ]:
trainForLaughter(number_of_samples=100, epochs=100)

In [ ]:
trainForLaughter(number_of_samples=999999, epochs=1)

training on laughter and not laughter
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt


In [ ]:
trainForLaughter(number_of_samples=999999, epochs=10)

In [ ]:
trainForLaughter(number_of_samples=999999, epochs=100)

In [ ]:
trainForLaughter(number_of_samples=100, epochs=1)

In [ ]:
trainForLaughter(number_of_samples=100, epochs=10)